In [4]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator
    )

from panda_station import *

In [14]:
builder = DiagramBuilder()
station = builder.AddSystem(PandaStation())
station.setup_from_file("directives/three_tables.yaml")
station.add_panda_with_hand()
station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"),
                           RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/meat_can.sdf"),
                           RigidTransform(RotationMatrix(), [0.5, 0.2, 0.2]))
station.finalize()
plant, scene_graph = station.get_plant_and_scene_graph()

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(5.0)
meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.
